In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sys
sys.path.append("../GilesCode")
from helper import *
from helper2_dep import *
from helper2_indep import *
from helper2_shapley_sampling import *
from helper4_kshap import *
import matplotlib.pyplot as plt
from scipy import stats

from os.path import join
import warnings 

warnings.filterwarnings('ignore')


In [56]:
# Load data
np.random.seed(1)
data = pd.read_csv('../Data/brca_small.csv')
X = data.values[:, :-1][:,:25]
Y = data.values[:, -1]

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=100, random_state=0)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=100, random_state=1)

# Normalize
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_val = (X_val - mean) / std
X_test = (X_test - mean) / std


In [89]:

def fit_model(x, y, x_val, y_val):
    # Cross validate for C
    C_list = np.arange(0.1, 1.0, 0.05)
    best_loss = np.inf
    best_C = None

    for C in C_list:
        # Fit model
        model = LogisticRegression(C=C, penalty='l1', multi_class='multinomial',
                                   solver='saga', max_iter=20000)
        model.fit(x, y)

        # Calculate loss
        val_loss = log_loss(y_val, model.predict_proba(x_val))

        # See if best
        if val_loss < best_loss:
            best_loss = val_loss
            best_C = C
            
    # Train model with all data
    model = LogisticRegression(C=best_C, penalty='l1', multi_class='multinomial',
                               solver='saga', max_iter=10000)
    model.fit(np.concatenate((x, x_val), axis=0),
              np.concatenate((y, y_val), axis=0))
    
    return model

# Train model
model = fit_model(X_train, Y_train, X_val, Y_val)

# OK, now let's get a gradient and hessian

d = X_train.shape[1]

BETA = model.coef_
A = model.intercept_.reshape(4,1)


def modelf(x):
    yhat = np.exp(A+np.dot(BETA,x.T))
    #return yhat.item() if x.shape[0]==1 else yhat
    return yhat[1]/np.sum(yhat)


def modelg(x):
    yhat = np.exp(A+np.dot(BETA,x.T))
    yhat = yhat/np.sum(yhat)
    
    return BETA[1]*yhat[1] - yhat[1]*np.dot(yhat.T,BETA)
    
def modelH(x):
    yhat = np.exp(A+np.dot(BETA,x.T))
    yhat = yhat/np.sum(yhat)
    
    return yhat[1]*(np.outer(BETA[1],BETA[1].T) -
        np.dot(np.dot(BETA.T,np.diagflat(yhat)),BETA) +
        np.outer(np.dot(yhat.T,BETA),np.dot(yhat.T,BETA)))

xloc = X_train[1].reshape(1,-1)
modelf(xloc)
gradient = modelg(xloc).T
hessian = modelH(xloc) # Not H?

In [99]:
independent_features = True
obj_ss = cv_shapley_sampling(modelf, X_train, xloc, 
                        independent_features,
                        gradient, hessian,
                        M=100, n_samples_per_perm=10) # M is number of permutations
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_ss

order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests, 0)**2**2)[order])) # Variance reductions


[52. 13.  0. 39. 46. 44.  8. 13. 29. 41. 29. 18.  0.  0. 31. 51.  0. 32.
  2.  0.  0.  0. 21. 18. 11.]


In [59]:
independent_features = True
obj_kshap = cv_kshap(modelf, X_train, xloc, 
            independent_features,
            gradient, hessian,
            var_method="wls",
            M=1000, n_samples_per_perm=10)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap

order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(corr_ests**2)[order])) # Variance reductions

[66. 61. 64. 69. 68. 46. 55. 32. 49. 54. 63. 74. 67. 49. 69. 52. 64. 63.
 58. 65. 73. 50. 53. 57. 60.]


Dependent features

In [60]:
feature_means = np.mean(X_train, axis=0)
cov_mat = np.cov(X_train, rowvar=False)

# Now to set up D matrices
u, s, vh = np.linalg.svd(cov_mat, full_matrices=True)
K = 10000
s_max = s[0]
min_acceptable = s_max/K
s2 = np.copy(s)
s2[s <= min_acceptable] = min_acceptable
cov2 = np.matmul(u, np.matmul(np.diag(s2), vh))

M_linear = 1000 # 10 seconds/1000 perms or so
D_matrices = make_all_lundberg_matrices(M_linear, cov2)

In [61]:
independent_features = False
shap_CV_true_dep = linear_shap_vals(xloc, D_matrices, feature_means, gradient)
obj_dep = cv_shapley_sampling(modelf, X_train, xloc,
                    independent_features,
                    gradient,
                    shap_CV_true=shap_CV_true_dep, # Equivalently, can give D_matrices instead
                    M=50,n_samples_per_perm=1,
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(corr_ests**2)[order])) # Variance reductions

[15. 10.  8.  9. 29. 27. 29. 31. 40. 15. 17. 41. 28. 12. 16. 20. 29. 31.
 32. 23.  9. 21. 13. 30. 38.]


In [62]:
np.random.seed(2)
independent_features = False
obj_kshap_dep = cv_kshap(modelf, X_train, xloc,
                    independent_features,
                    gradient,
                    shap_CV_true=shap_CV_true_dep,
                    M=1000,n_samples_per_perm=10, var_method="wls",
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(corr_ests**2)[order])) # Variance reductions

[21. 18. 21. 18. 24. 26. 28. 24. 24. 30. 21. 25. 28. 21. 21. 27. 23. 26.
 19. 15. 19. 17. 24. 26. 21.]
